## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-04-05-53-37 +0000,nyt,House Republicans Subpoena Jack Smith Over Pro...,https://www.nytimes.com/2025/12/03/us/politics...
1,2025-12-04-05-34-00 +0000,wsj,BOJ’s Ueda Says Unsure How Many More Rate Hike...,https://www.wsj.com/articles/bojs-ueda-says-un...
2,2025-12-04-05-25-21 +0000,bbc,Israel identifies hostage remains as Thai nati...,https://www.bbc.com/news/articles/cd6x2j4g3z7o...
3,2025-12-04-05-22-42 +0000,bbc,'The apprehension is palpable': Minnesota's So...,https://www.bbc.com/news/articles/c8dy1613j2ro...
4,2025-12-04-05-21-22 +0000,bbc,New mayoral elections to be delayed in two are...,https://www.bbc.com/news/articles/c5yjg2r7z9zo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2368/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
6,trump,50
28,new,19
12,he,15
136,ukraine,12
57,year,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
154,2025-12-03-22-05-00 +0000,wsj,"Rep. Elise Stefanik, a top Trump ally, critici...",https://www.wsj.com/politics/elise-stefanik-mi...,101
78,2025-12-04-00-05-45 +0000,nyt,Trump Faces Choices on Russia-Ukraine Talks as...,https://www.nytimes.com/2025/12/03/us/politics...,96
311,2025-12-03-13-18-46 +0000,bbc,Trump says he does not want Somalis in US as I...,https://www.bbc.com/news/articles/c208x9v68w3o...,88
76,2025-12-04-00-11-47 +0000,nyt,Trump Administration Begins Immigration Operat...,https://www.nytimes.com/2025/12/03/us/border-p...,87
130,2025-12-03-22-33-23 +0000,bbc,Trump administration to roll back fuel economy...,https://www.bbc.com/news/articles/cly13we8zxeo...,84


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
154,101,2025-12-03-22-05-00 +0000,wsj,"Rep. Elise Stefanik, a top Trump ally, critici...",https://www.wsj.com/politics/elise-stefanik-mi...
83,78,2025-12-04-00-00-00 +0000,wsj,Defense Secretary Pete Hegseth shocked officia...,https://www.wsj.com/politics/national-security...
42,56,2025-12-04-02-26-10 +0000,48hills,Under the Stars: Fresh new music nods to wrap ...,https://48hills.org/2025/12/under-the-stars-fr...
306,54,2025-12-03-13-33-53 +0000,nypost,Arrogant Putin sees ‘no point in making any se...,https://nypost.com/2025/12/03/world-news/putin...
43,47,2025-12-04-02-17-53 +0000,nypost,"Family of boy, 11, allegedly killed by illegal...",https://nypost.com/2025/12/03/us-news/family-o...
48,46,2025-12-04-01-55-16 +0000,nypost,Halle Berry bashes Gov. Gavin Newsom at NYT su...,https://nypost.com/2025/12/03/us-news/halle-be...
235,38,2025-12-03-18-05-36 +0000,nypost,Private employers cut 32K jobs last month — hi...,https://nypost.com/2025/12/03/business/private...
167,36,2025-12-03-21-20-09 +0000,nypost,MyPillow CEO Mike Lindell files paperwork to c...,https://nypost.com/2025/12/03/us-news/mypillow...
194,34,2025-12-03-20-27-24 +0000,nypost,"Pentagon watchdog finds Hegseth put US troops,...",https://nypost.com/2025/12/03/us-news/pentagon...
378,33,2025-12-03-06-31-16 +0000,latimes,Simi Valley doctor and wife were fatally shot ...,https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
